In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Load data

In [3]:
import sys
sys.path.append("runormas/")

In [4]:
from modules.data.read import DataReader

/home/user/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/iterator_ops.py:546: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class IteratorBase(collections.Iterator, trackable.Trackable,
/home/user/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:106: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class DatasetV2(collections.Iterable, tracking_base.Trackable,


In [147]:
reader = DataReader(
    path="data/public_test/",
    output_dir="data/public_test_v13",
    tokenizer_name="sberbank-ai/rugpt3xl",
    part="test",
    local_rank=0,
    word_size=1,
    window_size=40
)

Add answer_sep: <answer>
Add start_sep <start>
Add start_sep <end>


In [148]:
reader.prc(is_save=False)

Making raw files...: 100%|██████████| 536/536 [00:09<00:00, 57.14it/s]


In [118]:
from modules.utils import get_all_files_from_dir

In [227]:
files = get_all_files_from_dir("test_pred/v14_90k_fixed/")

In [228]:
len(files)

4906

In [229]:
baseline_preds = defaultdict(dict)
fixed_preds = defaultdict(dict)
errors = []
for fn in files:
    n_key = os.path.basename(fn)
    part = "named" if "named" in fn else "generic"
    key = n_key[:-5]
    baseline_preds[part][key] = []
    fixed_preds[part][key] = []
    with open(fn) as file:
        baseline_preds[part][key] = file.read().strip().split("\n")
    fn = os.path.join("test_pred/v14_130k_finetune/", part, n_key)
    if os.path.exists(fn):
        try:
            with open(fn) as file:
                fixed_preds[part][key] = file.read().strip().split("\n")
        except:
            print("error at", fn)
    else:
        fixed_preds[part][key] = baseline_preds[part][key]
        errors.append(fn)
    for idx in range(len(fixed_preds[part][key])):
        if len(fixed_preds[part][key]) == idx:
            fixed_preds[part][key].append(baseline_preds[part][key][idx])

Here 10 files was not predicted because trancated while distributed generation

In [230]:
no_beams_preds = baseline_preds

### Post-process

In [18]:
from pymorphy2 import MorphAnalyzer

In [ ]:
morph = MorphAnalyzer()

In [218]:
from string import punctuation
punctuation = set(punctuation + "«»")


def fix_verbs(data_part, name, idx, fixed_preds, ann):
    if len(fixed_preds[data_part][name][idx].split()) == 1 and data_part == "generic":
        parsed = morph.parse(fixed_preds[data_part][name][idx])[0]
        if parsed.tag.POS in ["VERB", "INFN"]:
            fixed_preds[data_part][name][idx] = parsed.normal_form
        
def fix_title(data_part, name, idx, fixed_preds, ann):
    if ann.istitle():
        fixed_preds[data_part][name][idx] = fixed_preds[data_part][name][idx].title()
        return int(not fixed_preds[data_part][name][idx].istitle())
    return 0


def fix_upper_names(data_part, name, idx, fixed_preds, ann, ends=["ом", "а", "у", "е"]):
    for x in ends:
        if ann.endswith(x) and ann.replace(x, "").lower() == fixed_preds[data_part][name][idx]:
            if ann.replace(x, "").isupper():
                # print(fixed_preds[data_part][name][idx], ann)
                fixed_preds[data_part][name][idx] = fixed_preds[data_part][name][idx].upper()
                return 1
    return 0


def fix_e(data_part, name, idx, fixed_preds, ann):
    return 0
    if "ё" in fixed_preds[data_part][name][idx] and "ё" not in ann and data_part == "generic":
        fixed_preds[data_part][name][idx] = fixed_preds[data_part][name][idx].replace("ё", "е")
        return 1
    return 0


def fix_special_tokens(data_part, name, idx, fixed_preds, reader):
    if sum([x in fixed_preds[data_part][name][idx] for x in ["<s>", "</s>", reader.answer_sep, reader.start_sep]]):
        for x in ["<s>", "</s>", reader.answer_sep, reader.start_sep]:
            fixed_preds[data_part][name][idx] = fixed_preds[data_part][name][idx].replace(f" {x} ", " ")
            fixed_preds[data_part][name][idx] = fixed_preds[data_part][name][idx].replace(f"{x} ", " ")
            fixed_preds[data_part][name][idx] = fixed_preds[data_part][name][idx].replace(f" {x}", " ")
            fixed_preds[data_part][name][idx] = fixed_preds[data_part][name][idx].replace(f" {x}", " ")
            fixed_preds[data_part][name][idx] = fixed_preds[data_part][name][idx].replace(f"{x}", " ").strip()
        return 1
    return 0


def fix_words_count(data_part, name, idx, fixed_preds, ann):
    if len(fixed_preds[data_part][name][idx].split()) != len(ann.split()):
        # print(ann, fixed_preds[data_part][name][idx], baseline_preds[data_part][name][idx], sep=" | ")
        fixed_preds[data_part][name][idx] = baseline_preds[data_part][name][idx]
        return 1
    return 0


def fix_punct(data_part, name, idx, fixed_preds, ann):
    gen_res = fixed_preds[data_part][name][idx]
    is_err = False
    while len(gen_res) and not gen_res.lower().startswith(ann[0].lower()) and gen_res[0] in punctuation:
        gen_res = gen_res[1:]
        is_err = True
    while len(gen_res) and not gen_res.lower().endswith(ann[-1].lower()) and gen_res[-1] in punctuation:
        gen_res = gen_res[:-1]
        is_err = True
    if len(gen_res):
        fixed_preds[data_part][name][idx] = gen_res
    if not fixed_preds[data_part][name][idx].lower().startswith(ann[0].lower()):
        is_err = True
        fixed_preds[data_part][name][idx] = baseline_preds[data_part][name][idx]
    return is_err


def fix_is_title(data_part, name, idx, fixed_preds, ann):
    gen_res = fixed_preds[data_part][name][idx]
    try:
        if not gen_res[0].istitle() and ann[0].istitle():
            fixed_preds[data_part][name][idx] = gen_res[0].title() + gen_res[1:]
            return 1
    except:
        print(data_part, ann, fixed_preds[data_part][name][idx], baseline_preds[data_part][name][idx], sep="|")
    return 0


def fix_end_sym(data_part, name, idx, fixed_preds, ann):
    if ann[-1] in punctuation and fixed_preds[data_part][name][idx][-1] not in punctuation:
        fixed_preds[data_part][name][idx] += ann[-1]
        return 1
    return 0

In [237]:
fixed_preds_anns = 0
fixed_predicted_anns = defaultdict(dict)
errors = 0
is_title_errors = 0
compare_baseline = defaultdict(dict)
token_errors = 0
errors_len = 0
intersect_bound = 0.6
intersect_errors = 0
e_errors = 0
is_generate = False
upper_errors = 0
word_count_err = 0
start_sym_err = 0
bound_len = 0.5
char_len_errors = 0
is_title_errors = 0
end_sym_errors = 0
for data_part in reader.lm_prefixes:

    for name in tqdm(reader.lm_prefixes[data_part], total=len(reader.lm_prefixes[data_part])):
        fixed_predicted_anns[data_part][name] = []
        compare_baseline[data_part][name] = []
        for idx, lm_prefix in enumerate(reader.lm_prefixes[data_part][name]):
            text = reader.texts[data_part][name]
            ann = reader.anns[data_part][name][idx]
            start, stop = list(map(int, ann.split()))
            ann = text[start:stop].strip()
            try:
                _ = baseline_preds[data_part][name][idx]
            except:
                baseline_preds[data_part][name].append(ann)
            try:
                _ = fixed_preds[data_part][name][idx]
            except:
                fixed_preds[data_part][name].append(no_beams_preds[data_part][name][idx])
            if not len(fixed_preds[data_part][name][idx]):
                fixed_preds[data_part][name][idx] = no_beams_preds[data_part][name][idx]
            if not len(fixed_preds[data_part][name][idx]):
                fixed_preds[data_part][name][idx] = baseline_preds[data_part][name][idx]
            if not len(fixed_preds[data_part][name][idx]):
                fixed_preds[data_part][name][idx] = ann
            token_errors += fix_special_tokens(data_part, name, idx, fixed_preds, reader)
            is_title_errors += fix_title(data_part, name, idx, fixed_preds, ann)
            upper_errors += fix_upper_names(data_part, name, idx, fixed_preds, ann)
            e_errors += fix_e(data_part, name, idx, fixed_preds, ann)
            is_title_errors + fix_is_title(data_part, name, idx, fixed_preds, ann)
            start_sym_err += fix_punct(data_part, name, idx, fixed_preds, ann)
            end_sym_errors + fix_end_sym(data_part, name, idx, fixed_preds, ann)
            word_count_err += fix_words_count(data_part, name, idx, fixed_preds, ann)
            if len(fixed_preds[data_part][name][idx]) / len(ann) < bound_len:
                char_len_errors += 1
                fixed_preds[data_part][name][idx] = baseline_preds[data_part][name][idx]
            if fixed_preds[data_part][name][idx] == ann:
                fixed_preds_anns += 1
                fixed_predicted_anns[data_part][name].append(
                    {"ann": ann, "gen_res": fixed_preds[data_part][name][idx],
                     "text": text, "lm_prefix": lm_prefix})
            
            intersect = len(set(fixed_preds[data_part][name][idx]).intersection(ann))
            intersect /= max(1, len(set(fixed_preds[data_part][name][idx])))
            if intersect < intersect_bound:
                intersect_errors += 1
            
            if intersect < intersect_bound and is_generate:
                # print(ann, fixed_preds[data_part][name][idx], baseline_preds[data_part][name][idx], sep="|")
                gen_res = generate(model, lm_prefix, num_beams=None, do_sample=False)
                gen_res = gen_res.split(reader.answer_sep)
                if len(gen_res) == 1:
                    gen_res = text[start:stop].strip()
                else:
                    gen_res = gen_res[1].strip()
                if len(gen_res.split()) == 1 and "област" in gen_res:
                    gen_res = "область"
                    
                fixed_preds[data_part][name][idx] = gen_res

                fix_special_tokens(data_part, name, idx, fixed_preds, reader)
                fix_title(data_part, name, idx, fixed_preds, ann)
                fix_upper_names(data_part, name, idx, fixed_preds, ann)
                fix_e(data_part, name, idx, fixed_preds, ann)
                word_count_err += fix_words_count(data_part, name, idx, fixed_preds, ann)
                is_title_errors + fix_is_title(data_part, name, idx, fixed_preds, ann)
                start_sym_err += fix_punct(data_part, name, idx, fixed_preds, ann)
                end_sym_errors + fix_end_sym(data_part, name, idx, fixed_preds, ann)
                intersect = len(set(fixed_preds[data_part][name][idx]).intersection(ann))
                intersect /= max(1, len(set(fixed_preds[data_part][name][idx])))
                if intersect < 0.6:
                    print(data_part, ann, fixed_preds[data_part][name][idx], baseline_preds[data_part][name][idx], sep="|")
            elif intersect < intersect_bound:
                fixed_preds[data_part][name][idx] = baseline_preds[data_part][name][idx]
            fix_verbs(data_part, name, idx, fixed_preds, ann)
            if fixed_preds[data_part][name][idx] != baseline_preds[data_part][name][idx]:
                compare_baseline[data_part][name].append(
                    {"ann": ann,
                     "gen_res": fixed_preds[data_part][name][idx],
                     "text": text,
                     "lm_prefix": lm_prefix,
                     "baseline": baseline_preds[data_part][name][idx]
                    })

100%|██████████| 536/536 [00:02<00:00, 235.16it/s]


In [238]:
{
    "is_title_errors": is_title_errors,
    "token_errors": token_errors,
    "errors_len": errors_len,
    "intersect_errors": intersect_errors,
    "e_errors": e_errors,
    "upper_errors": upper_errors,
    "word_count_err": word_count_err,
    "char_len_errors": char_len_errors,
    "start_sym_err": start_sym_err,
    "is_title_errors": is_title_errors,
    "end_sym_errors": end_sym_errors
}

{'is_title_errors': 0,
 'token_errors': 0,
 'errors_len': 0,
 'intersect_errors': 35,
 'e_errors': 0,
 'upper_errors': 0,
 'word_count_err': 0,
 'char_len_errors': 0,
 'start_sym_err': 6,
 'end_sym_errors': 0}

#### fix ws

In [239]:
errors = 0
for data_part in ["named"]:
    for name in list(reader.lm_prefixes[data_part]):
        for idx, lm_prefix in enumerate(reader.lm_prefixes[data_part][name]):
            text = reader.texts[data_part][name]
            ann = reader.anns[data_part][name][idx]
            start, stop = list(map(int, ann.split()))
            ann = text[start:stop].strip()
            gen_res = fixed_preds[data_part][name][idx]
            # if sum([x.istitle() for x in gen_res.split()]) != sum([x.istitle() for x in ann.split()]):
            #     errors += 1
            if ann.count(" ") != fixed_preds[data_part][name][idx].count(" "):
                #print(data_part, ann, fixed_preds[data_part][name][idx], baseline_preds[data_part][name][idx], sep="|")
                #continue
                words = ann.split()
                text_ann = ann
                ws = []
                gen_words = gen_res.split()
                new_gen_res = gen_words[0]
                assert len(words) == len(gen_words)
                for idx_, (word, gen_word) in enumerate(zip(words[:-1], gen_words[1:])):
                    pos = text_ann.find(word)
                    while pos < len(text_ann) and not text_ann[pos:].startswith(words[idx_ + 1]):
                        pos += 1
                    new_gen_res += text_ann[len(word):pos] + gen_word
                    text_ann = text_ann[pos:]
                fixed_preds[data_part][name][idx] = new_gen_res
                errors += 1
                # print(ann.count(" "), ann.split())
                # print(data_part, ann, fixed_preds[data_part][name][idx], baseline_preds[data_part][name][idx], sep="|")
    break

### Save predictions

In [240]:
path = "test_pred/v14_130k_finetune_fixed/"

In [241]:
for data_part in fixed_preds:
    store_dir = os.path.join(path, data_part)
    if not os.path.exists(store_dir):
        os.makedirs(store_dir, exist_ok=True)
    names = list(fixed_preds[data_part].keys())

    for name in tqdm(names, total=len(names), leave=True, desc=f"Predict on {0}"):
        with open(os.path.join(store_dir, f"{name}.norm"), 'w', encoding='utf-8') as file:
            for pred in fixed_preds[data_part][name]:
                file.write(f"{pred}\n")

Predict on 0: 100%|██████████| 536/536 [00:01<00:00, 387.61it/s]
